# 1. im2colの実装の理解
numpyのテクニックが豊富に使われていると思われる（少なくとも自分にはそう思える）im2col/col2imの実装を調べて、numpyについて理解を深める。
im2colとcol2imのコードを分解してステップバイステップで実行結果を見ていくことで、実装の細かな所を理解してnumpyのテクニックについて習得する。

## 1-1. 全体的な設定

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from common.util import *

from my_common.util import init_sample_matrix

In [2]:
#入力画像の高さ、幅、および、チャネル数。画像の枚数
H  = 4
W  = 5
C  = 1 #mnistはチャンネル数1のようなので、このサンプルテストもそれに合わせる(7.4.2と同じ)
N  = 2 #入力画像の枚数

#poolingレイヤの基本パラメータ
pool_h = 2
pool_w = 2
stride = 2
pad    = 0

#出力データ（画像）の高さと幅（自動的に計算される。）
out_h = int(1 + (H - pool_h) / stride)
out_w = int(1 + (W - pool_w) / stride)

print("出力画像の高さ=%d と 幅=%d" % (out_h ,out_w))

出力画像の高さ=2 と 幅=2


## 1-3. 入力画像xの用意

In [3]:
#xの用意(mnistはチャンネル数1のようなので、このテストもそれに合わせる)
print("=== preparing of x===")
x1 = init_sample_matrix(filter_num = 1, channel=C, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x2 = init_sample_matrix(filter_num = 2, channel=C, height=H, width=W)
x = np.array([[x1],[x2]])
print(x.shape)
print(x)

=== preparing of x===
(2, 1, 4, 5)
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]


## 1-4. まずはim2colを実行してみる

In [4]:
col = im2col(x, pool_h, pool_w, stride, pad)
print(col)

[[1111. 1112. 1121. 1122.]
 [1113. 1114. 1123. 1124.]
 [1131. 1132. 1141. 1142.]
 [1133. 1134. 1143. 1144.]
 [2111. 2112. 2121. 2122.]
 [2113. 2114. 2123. 2124.]
 [2131. 2132. 2141. 2142.]
 [2133. 2134. 2143. 2144.]]


テキストのＰ２２８の説明にあるとおり、プーリングの大きさで行ベクトルが切り出されていることがわかる。

## 1-5. step by stepで実行してみよう

In [5]:
print("im2colへの入力")
input_data = x
filter_h = 2
filter_w = 2
stride = 1
pad = 0

print("最初に出力データのサイズを計算する")
N, C, H, W = input_data.shape
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

print(x)
print(N)
print(C)
print(H)
print(W)
print(out_h)
print(out_w)


im2colへの入力
最初に出力データのサイズを計算する
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]
2
1
4
5
3
4


In [6]:
print("imgを計算する")
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
print(img)

imgを計算する
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]


結果は変わらないことになった。np.padについて詳しく性質を調べてみたので、そちらはnumpy_excersiceに記してある。要するにpad=0なので、どの次元に対してもパディングしないということ。

In [7]:
print("colの作成")
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
print(col)

colの作成
[[[[[[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]

    [[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]]


   [[[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]

    [[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]]]]




 [[[[[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]

    [[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]]


   [[[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]

    [[0. 0. 0. 0.]
     [0. 0. 0. 0.]
     [0. 0. 0. 0.]]]]]]


In [8]:
print(filter_h)
print(filter_w)
print(out_h)
print(out_w)
print(stride) #strideが1なので、結局飛ばしてアクセスはしない。
print(img)
print(img.shape)
print(col.shape) #(2, 1, 2, 2, 3, 4) →　画像枚数、チャネル枚数、フィルタの高さ、フィルタの幅、imgから抜き出した画像の高さ、imgから抜き出した画像の幅
for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        print("===============")
        x_max = x + stride*out_w
        print("x, xmax, y, y_max")
        print(x, x_max, y, y_max)
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
        print("img")
        print(img[:, :, y:y_max:stride, x:x_max:stride])
        print("col")
        print(col)
        
print("RESULT OF col")
print(col)

2
2
3
4
1
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]
(2, 1, 4, 5)
(2, 1, 2, 2, 3, 4)
x, xmax, y, y_max
0 4 0 3
img
[[[[1111 1112 1113 1114]
   [1121 1122 1123 1124]
   [1131 1132 1133 1134]]]


 [[[2111 2112 2113 2114]
   [2121 2122 2123 2124]
   [2131 2132 2133 2134]]]]
col
[[[[[[1111. 1112. 1113. 1114.]
     [1121. 1122. 1123. 1124.]
     [1131. 1132. 1133. 1134.]]

    [[   0.    0.    0.    0.]
     [   0.    0.    0.    0.]
     [   0.    0.    0.    0.]]]


   [[[   0.    0.    0.    0.]
     [   0.    0.    0.    0.]
     [   0.    0.    0.    0.]]

    [[   0.    0.    0.    0.]
     [   0.    0.    0.    0.]
     [   0.    0.    0.    0.]]]]]




 [[[[[2111. 2112. 2113. 2114.]
     [2121. 2122. 2123. 2124.]
     [2131. 2132. 2133. 2134.]]

    [[   0.    0.    0.    0.]
     [

In [9]:
print("transpose")
col = col.transpose(0, 4, 5, 1, 2, 3)
print(col)


transpose
[[[[[[1111. 1112.]
     [1121. 1122.]]]


   [[[1112. 1113.]
     [1122. 1123.]]]


   [[[1113. 1114.]
     [1123. 1124.]]]


   [[[1114. 1115.]
     [1124. 1125.]]]]



  [[[[1121. 1122.]
     [1131. 1132.]]]


   [[[1122. 1123.]
     [1132. 1133.]]]


   [[[1123. 1124.]
     [1133. 1134.]]]


   [[[1124. 1125.]
     [1134. 1135.]]]]



  [[[[1131. 1132.]
     [1141. 1142.]]]


   [[[1132. 1133.]
     [1142. 1143.]]]


   [[[1133. 1134.]
     [1143. 1144.]]]


   [[[1134. 1135.]
     [1144. 1145.]]]]]




 [[[[[2111. 2112.]
     [2121. 2122.]]]


   [[[2112. 2113.]
     [2122. 2123.]]]


   [[[2113. 2114.]
     [2123. 2124.]]]


   [[[2114. 2115.]
     [2124. 2125.]]]]



  [[[[2121. 2122.]
     [2131. 2132.]]]


   [[[2122. 2123.]
     [2132. 2133.]]]


   [[[2123. 2124.]
     [2133. 2134.]]]


   [[[2124. 2125.]
     [2134. 2135.]]]]



  [[[[2131. 2132.]
     [2141. 2142.]]]


   [[[2132. 2133.]
     [2142. 2143.]]]


   [[[2133. 2134.]
     [2143. 2144.]]]


   [[[2134. 

In [10]:
print("reshape(result)")
col = col.reshape(N*out_h*out_w, -1)
print(col)

reshape(result)
[[1111. 1112. 1121. 1122.]
 [1112. 1113. 1122. 1123.]
 [1113. 1114. 1123. 1124.]
 [1114. 1115. 1124. 1125.]
 [1121. 1122. 1131. 1132.]
 [1122. 1123. 1132. 1133.]
 [1123. 1124. 1133. 1134.]
 [1124. 1125. 1134. 1135.]
 [1131. 1132. 1141. 1142.]
 [1132. 1133. 1142. 1143.]
 [1133. 1134. 1143. 1144.]
 [1134. 1135. 1144. 1145.]
 [2111. 2112. 2121. 2122.]
 [2112. 2113. 2122. 2123.]
 [2113. 2114. 2123. 2124.]
 [2114. 2115. 2124. 2125.]
 [2121. 2122. 2131. 2132.]
 [2122. 2123. 2132. 2133.]
 [2123. 2124. 2133. 2134.]
 [2124. 2125. 2134. 2135.]
 [2131. 2132. 2141. 2142.]
 [2132. 2133. 2142. 2143.]
 [2133. 2134. 2143. 2144.]
 [2134. 2135. 2144. 2145.]]
